# Helper Function

In [7]:
from __future__ import print_function
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time, re, os
from datetime import datetime as dt
import pandas as pd


def run_seleniun_and_get_page_source():
    url = 'https://www.ycombinator.com/companies'
    options = webdriver.ChromeOptions()
    options.headless = True
    try:
        path = os.getenv('CHROMEDRIVER_HOME')
        driver = webdriver.Chrome(executable_path=path, chrome_options=options)
    except Exception as e:
        driver = webdriver.Chrome(ChromeDriverManager().install())

    driver.get(url)
    time.sleep(5)

    #scroll to the end of the page
    check_page_length = 0
    try:
        while True:
            page_len = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            time.sleep(0.5)

            if check_page_length == page_len:
                break
            check_page_length = page_len
    except:
        driver.close()

    selenium_web_content = BeautifulSoup(driver.page_source, 'lxml')
    get_company_list_block = selenium_web_content.find_all('a', class_='WxyYeI15LZ5U_DOM0z8F no-hovercard')
    
    return get_company_list_block, driver


def get_company_info(soup):
    info = {}

    company_data = soup.find('div',  class_='my-8 mb-4')
    info['link'] = company_data.find('div', class_='flex flex-row items-center px-3 leading-none').a['href']

    summary = soup.find('div',  class_='space-y-3')
    info['company_name'] = summary.h1.text
    info['short_description'] = summary.find('div',  class_='text-xl').text

    spans = summary.find_all('span',  class_='ycdc-badge')
    info['tags'] = [what.text.replace('Y Combinator Logo', '') for what in spans]
    info['description'] = soup.p.text
    info['company_socials'] = soup.find('div',  class_='space-x-2')
    
    spans = []

    i = 0
    for fact in soup.find('div', class_="space-y-0.5").find_all('span'):
        spans.append(fact.text)
        try:
            key_ = spans[i].lower().replace(' ', '_')[:-1]
            info[key_] = spans[i+1]
            i += 2
        except:
            pass

    info['company_socials'] = [a['href'] for a in soup.find('div',  class_='space-x-2').find_all('a')]

    return info


def get_founders_info(soup):
    founders_info = {}
    try:
        #? This implementation is for active founders
        founders = soup.find('div', class_='space-y-5')
        founders_info['active_founders'] = [name.div.text for name in founders.find_all('div', class_='leading-snug')]

        all_about = []
        for what in founders.find_all('div', class_='leading-snug'):
            about_founder = {}
            name = what.find('div', class_='font-bold').text
            about_founder['name'] = name
            about_founder['social_media_links'] = [link['href'] for link in what.find('div', class_='mt-1 space-x-2').find_all('a')]
            
            all_about.append(about_founder)

        founders_info['about_founders'] = all_about
        
    except:
        founders = soup.find('div', class_='space-y-4')
        founders_info = {}
        all_about = []
        for founder in founders.find_all('div', class_='leading-snug'):
            about_founder = {}
            name = founder.find('div', class_='font-bold').text
            founders_info['active_founders'] = [name]

            about_founder['name'] = name

            divs = [ what for what in founder.find_all('div')]
            about_founder['role'] = divs[1].text
            about_founder['social_media_links'] = [link['href'] for link in founder.find('div', class_='mt-1 space-x-2').find_all('a')]

            all_about.append(about_founder)
            
        founders_info['about_founders'] = all_about

    return founders_info


def scrape_info(link_href):
    main_url = 'https://www.ycombinator.com'
    url = main_url + link_href
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')

    company_all_info = get_company_info(soup)
    founder_info = get_founders_info(soup)
    company_all_info.update(founder_info)

    return company_all_info


def save_to_csv(scraped_info, savepath):
       df = pd.DataFrame(scraped_info)
       df = df[['company_name', 'link', 'short_description', 'tags',
              'company_socials', 'founded', 'team_size', 'location',
              'active_founders', 'about_founders', 'description']]
              
       df.to_csv(savepath, index=False)


# Get all links from Y-combinator
`run_seleniun_and_get_page_source`: Used to handle the dynamic scraping of the project. It scrolls the website till it reaches the end of the page. Afterward, beautifulsoup is used to extract all the links to the individual company's page.

In [8]:

main_url = 'https://www.ycombinator.com'

start = dt.now()
get_company_list_block, driver = run_seleniun_and_get_page_source()
driver.close()


print("Selenium closed, handing over to BeautifulSoup")

y_company_page_urls = [link['href'] for link in get_company_list_block]
length = len(y_company_page_urls)
print(f"Number of companies: {length}")

runtime = (dt.now() - start).total_seconds()
print(f'Total scroll runtime - {runtime}')

/var/folders/dd/p_l_68gj2hj_qyc4_9yblp580000gn/T/ipykernel_58152/810837057.py:16: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
/var/folders/dd/p_l_68gj2hj_qyc4_9yblp580000gn/T/ipykernel_58152/810837057.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, chrome_options=options)
/var/folders/dd/p_l_68gj2hj_qyc4_9yblp580000gn/T/ipykernel_58152/810837057.py:19: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=path, chrome_options=options)
/var/folders/dd/p_l_68gj2hj_qyc4_9yblp580000gn/T/ipykernel_58152/810837057.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Selenium closed, handing over to BeautifulSoup
Number of companies: 1000
Total scroll runtime - 40.743295


Do this for all the batches

In [ ]:
all_batches = ['W23', 'S22', 'W22', 'S21', 'W21', 'S20', 'W20', 'S19', 'W19', 'S18', 'W18', 'S17', 'W17', 'IK12', 'S16', 'W16', 'S15', 'W15', 'S14', 'W14', 'S13', 'W13', 'S12', 'W12', 'S11', 'W11', 'S10', 'W10', 'S09', 'W09', 'S08', 'W08', 'S07', 'W07', 'S06', 'W06', 'S05']

Remove known faulty company websites

In [9]:
known_faulty_websites = ['/companies/11874']
for known_faulty_website in known_faulty_websites:
    try:
        y_company_page_urls.remove(known_faulty_website)
    except:
        # already removed
        pass
    
print(len(y_company_page_urls))

999


# Scraping using multi-threading
Multithreading gives a faster and more efficient reuslt, the script ran for about `~62 secs` as compared the the first appoach.

`Note:` The runtime can be faster or slower depending on internet connectivity.

In [11]:
import concurrent.futures as cf


start_thread = dt.now()
m_companies = []
with cf.ThreadPoolExecutor() as exc:
    results = exc.map(scrape_info, y_company_page_urls)

    for result in results:
        m_companies.append(result)

runtime_thread = (dt.now() - start_thread).total_seconds()
print(f'Total runtime - {runtime_thread}')
print(m_companies[0]['about_founders'])

Total runtime - 36.798173
[{'name': 'Nathan Blecharczyk', 'social_media_links': ['https://twitter.com/nathanblec', 'https://www.linkedin.com/in/blecharczyk/']}, {'name': 'Brian Chesky', 'social_media_links': ['https://twitter.com/bchesky']}, {'name': 'Joe Gebbia', 'social_media_links': ['https://twitter.com/jgebbia', 'https://www.linkedin.com/in/brianchesky']}]


# Save as CSV

In [12]:
import pandas as pd

save_to_csv(scraped_info=m_companies, savepath='ycombinator.csv')
       